<a href="https://colab.research.google.com/github/staocube88/AI-Model-Training/blob/master/Optimized_FAW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
from google.colab import drive

# 1. Install YOLOv8 (Ultralytics)
!pip install ultralytics -q
# 2. Mount Google Drive
drive.mount('/content/drive')

# --- Define Paths ---
# Source: Your top-level folder in Google Drive
DRIVE_ROOT = '/content/drive/MyDrive/FAW_Dataset'

# Destination: Colab's fast local disk
LOCAL_ROOT = '/content/FAW_Dataset'

print(f"Copying {DRIVE_ROOT} to local storage {LOCAL_ROOT}...")

# Remove the destination directory if it already exists
if os.path.exists(LOCAL_ROOT):
    print(f"Removing existing directory: {LOCAL_ROOT}")
    shutil.rmtree(LOCAL_ROOT)

# Copy the entire directory structure (including 'train', 'test', and their subfolders)
shutil.copytree(DRIVE_ROOT, LOCAL_ROOT)

print("✅ Initial Copy Complete! All subsequent operations will use local disk.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00
Mounted at /content/drive
Copying /content/drive/MyDrive/FAW_Dataset to local storage /content/FAW_Dataset...
✅ Initial Copy Complete! All subsequent operations will use local disk.


In [2]:
# --- Configuration ---
# NOTE: This is the root where 'train' and 'test' folders reside
LOCAL_ROOT = '/content/FAW_Dataset'

# Your classification folders (keys) and their corresponding YOLO class IDs (values)
CLASS_MAPPING = {
    # FIX: These keys MUST EXACTLY match your folder names:
    "FAW_larva": 0,
    "FAW_damage": 1,
    "healthy_crops": 2 # Assuming this is your third class
}

# --- Define New YOLO Structure (Output) ---
# We create ONE merged folder for all data (before the final train/val split)
YOLO_ROOT = os.path.join(LOCAL_ROOT, 'YOLO_FAW_Dataset_MERGED')
os.makedirs(os.path.join(YOLO_ROOT, 'images'), exist_ok=True)
os.makedirs(os.path.join(YOLO_ROOT, 'labels'), exist_ok=True)

print("Starting conversion and merging of train/test splits...")
total_files = 0

# Iterate over the primary splits: 'train' and 'test'
for split_folder in ['train', 'test']:
    split_path = os.path.join(LOCAL_ROOT, split_folder)

    if not os.path.isdir(split_path):
        print(f"Skipping primary split folder: {split_path} (Not Found)")
        continue

    # Iterate over the classification folders inside the split (e.g., train/FAW_Larva)
    for class_name, class_id in CLASS_MAPPING.items():
        # This is the path to the images: e.g., /content/drive/MyDrive/FAW_Dataset/train/FAW_Larva
        class_folder = os.path.join(split_path, class_name)

        if not os.path.isdir(class_folder):
            print(f"  Skipping class folder: {class_folder} (Not Found)")
            continue

        # Process all image files in that specific class folder
        for filename in os.listdir(class_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                total_files += 1
                img_path = os.path.join(class_folder, filename)

                # 1. Create a unique filename for the image and label
                # This ensures images with the same name (e.g., '1.jpg') from 'train' and 'test' don't overwrite each other
                base_name = os.path.splitext(filename)[0]
                unique_filename = f"{split_folder}_{class_name}_{base_name}{os.path.splitext(filename)[1]}"
                unique_label_name = f"{split_folder}_{class_name}_{base_name}.txt"

                # 2. Copy Image to the new 'images' directory
                new_img_path = os.path.join(YOLO_ROOT, 'images', unique_filename)
                shutil.copy(img_path, new_img_path)

                # 3. Create the YOLO .txt label file
                label_path = os.path.join(YOLO_ROOT, 'labels', unique_label_name)

                # YOLO format: [class_id] [x_center] [y_center] [width] [height]
                yolo_line = f"{class_id} 0.5 0.5 1.0 1.0\n"

                with open(label_path, 'w') as f:
                    f.write(yolo_line)

print(f"\n✅ Conversion & Merging Complete! Processed {total_files} files.")
print(f"New MERGED YOLO dataset created at: {YOLO_ROOT}")

Starting conversion and merging of train/test splits...

✅ Conversion & Merging Complete! Processed 3443 files.
New MERGED YOLO dataset created at: /content/FAW_Dataset/YOLO_FAW_Dataset_MERGED


In [3]:
# Install the library designed for splitting image folders
!pip install split-folders -q
import splitfolders

In [4]:
# Install necessary libraries for splitting and YAML creation (run this cell if not done)
!pip install scikit-learn pyyaml -q
import os
import shutil
import glob
import yaml
from sklearn.model_selection import train_test_split

# --- Configuration ---
# Input: The merged folder you just created
YOLO_ROOT_INPUT = '/content/FAW_Dataset/YOLO_FAW_Dataset_MERGED'
# Output: The final structured folder for training
YOLO_ROOT_OUTPUT = '/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT'

# Define new final split paths and create directories
for folder in ['train/images', 'train/labels', 'val/images', 'val/labels']:
    os.makedirs(os.path.join(YOLO_ROOT_OUTPUT, folder), exist_ok=True)

# 1. Get a list of all image paths from the merged folder
all_image_paths = glob.glob(os.path.join(YOLO_ROOT_INPUT, 'images', '*.jpg'))

if not all_image_paths:
    print("❌ ERROR: No images found in the merged input folder. Check path and file extensions.")
    exit()

# 2. Split the paths into train (80%) and validation (20%) sets
train_paths, val_paths = train_test_split(
    all_image_paths,
    test_size=0.2,
    random_state=42
)

# 3. Function to copy files and their corresponding labels
def copy_split_files(file_paths, split_name):
    print(f"Copying {len(file_paths)} files to {split_name}...")
    img_target = os.path.join(YOLO_ROOT_OUTPUT, split_name, 'images')
    label_target = os.path.join(YOLO_ROOT_OUTPUT, split_name, 'labels')

    for img_path in file_paths:
        # Define image and label names
        filename = os.path.basename(img_path)
        base_name = os.path.splitext(filename)[0]
        label_filename = base_name + '.txt'

        # Source paths for image and label (from the MERGED folder)
        label_path_src = os.path.join(YOLO_ROOT_INPUT, 'labels', label_filename)

        # Copy image and label to the SPLIT folder
        shutil.copy(img_path, os.path.join(img_target, filename))
        shutil.copy(label_path_src, os.path.join(label_target, label_filename))

# Copy the files
copy_split_files(train_paths, 'train')
copy_split_files(val_paths, 'val')

print(f"\n✅ Data Splitting Complete! Train: {len(train_paths)} files, Val: {len(val_paths)} files.")

Copying 2718 files to train...
Copying 680 files to val...

✅ Data Splitting Complete! Train: 2718 files, Val: 680 files.


In [5]:
# Install pyyaml if needed
!pip install pyyaml -q
import yaml

# --- Updated Configuration for YOLOv8 ---
# Define the path for the split dataset root
LOCAL_SPLIT_ROOT = '/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT' # Define the variable here
LOCAL_DATA_YAML_PATH = os.path.join(LOCAL_SPLIT_ROOT, 'data.yaml')

yaml_content = {
    # Path where YOLO will look for the train/val folders (local disk)
    'path': LOCAL_SPLIT_ROOT,
    'train': 'train/images',
    'val': 'val/images',
    'nc': 3,
    'names': ['FAW_larva', 'FAW_damage', 'healthy_crops']
}

# Write the content to a NEW file in the local directory
with open(LOCAL_DATA_YAML_PATH, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

print(f"\n✅ Updated data.yaml created locally at: {LOCAL_DATA_YAML_PATH}")


✅ Updated data.yaml created locally at: /content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT/data.yaml


In [6]:
from ultralytics import YOLO

# Install Ultralytics again if your runtime was disconnected after the copy
# !pip install ultralytics -q

model = YOLO('yolov8n.pt')

# Note the new path here
FAST_DATA_YAML_PATH = '/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT/data.yaml'

print("\n🚀 Starting FAST YOLOv8 Training...")

results = model.train(
    data=FAST_DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    name='FAW_Object_Detection_Fast',
    workers=8, # Enables parallel data loading (critical for speed)
    batch=32 # Try to maximize this value for GPU efficiency
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

🚀 Starting FAST YOLOv8 Training...
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015,

In [7]:
from ultralytics import YOLO

# 1. Load the pre-trained model
model = YOLO('yolov8n.pt')

# 2. Train the model
print("\n🔥 Starting YOLOv8 Training...")
# Using the path defined in step 2
FINAL_DATA_YAML_PATH = '/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT/data.yaml'

results = model.train(
    data=FINAL_DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    name='FAW_Object_Detection_Final',
    patience=25 # Stop training early if validation metrics don't improve for 25 epochs
)
print("Training Complete. Weights saved.")

# 3. Export to ONNX (The Core Deliverable)
BEST_WEIGHTS_PATH = f'/content/runs/detect/FAW_Object_Detection_Final/weights/best.pt'

model_to_export = YOLO(BEST_WEIGHTS_PATH)

print("\n📦 Exporting model to ONNX format...")
model_to_export.export(
    format='onnx',
    simplify=True,
    dynamic=False,
    imgsz=640
)

print("\n🎉 Model Training and ONNX Export Finished!")
print(f"Your final deliverable model file is located within the runs folder: {os.path.dirname(BEST_WEIGHTS_PATH)}/best.onnx")


🔥 Starting YOLOv8 Training...
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/FAW_Dataset/YOLO_FAW_Dataset_SPLIT/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=FAW_Object_Detection_Final, nbs=64, nms=False, opset=None, optimize=False, optimi